In [36]:

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



import numpy as np
from glob import glob
import cv2 as cv
import os 
from os.path import join,basename
from natsort import natsorted 
import random




seq_len=64


paths={'saved_dire':join('datasets_features','smarthome','rgb','densenet101','Full'),
'rgb_raw':join('raw_dataset')
}

for key,value in paths.items():
    if not os.path.exists(value):
        os.makedirs(value)

#features_siz=2048 ## outshape from the resnetv2_50
features_siz=1024 ## outshape from the  densenet101


def features_extraction_model(resized_height=480,resized_width=640):
    ## defining the DenseNet121 model pretrained with imagenet dataset
    feature_extractor=tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',

)
    ##Defining the input size for single image                    
    inputs=tf.keras.Input((resized_height,resized_width,3))
    preprocessed=tf.keras.applications.densenet.preprocess_input(inputs)

    ## Give features for each image
    outputs=feature_extractor(preprocessed)
    ## defining feature extractor model
    model=tf.keras.Model(inputs,outputs,name='feature_extractor')
    return model






densenet=features_extraction_model()


def sample_frames(video_dire):
    cap=cv.VideoCapture(video_dire)
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    skip_frames=total_frames//seq_len


    frames=[]

    if skip_frames==0:

        while cap.isOpened():
            
            sucess,frame=cap.read()
            
            if not sucess:
                break
            frames.append(frame)
        return np.array(frames)


    if skip_frames==1:
        selected_frames=random.sample(range(total_frames),seq_len)
        selected_frames=sorted(selected_frames)
        for frame_number in selected_frames:
            cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
            sucess,frame=cap.read()
            frames.append(frame)
        return np.array(frames)

    
    if skip_frames>1:

        selected_frames=[i for i in range(0,total_frames,skip_frames)]

        if len(selected_frames) >seq_len:
            np.random.shuffle(selected_frames)
            selected_frames=selected_frames[:seq_len]
            selected_frames=sorted(selected_frames)
            for frame_number in selected_frames:
                cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
                sucess,frame=cap.read()
                frames.append(frame)
            
            return np.array(frames)


        else:
            for frame_number in selected_frames:
                
                cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
                sucess,frame=cap.read()
            
                frames.append(frame)

            return np.array(frames) 

        




def extract_features(video_dire): ## takes the directory of the video as input 

    frames=sample_frames(video_dire)

    print(frames.shape[0])

    features=densenet.predict(frames)

    

    len_features=features.shape[0]

    if len_features==seq_len:
        return features


    if len_features < seq_len:

        diff_frame=seq_len-len_features
        padd_arr=np.zeros((diff_frame,1024),dtype=np.float32)
        features=np.concatenate((features,padd_arr))
        return features

    
    




def create_data(class_idx,video_idx):

    
   

    classNames=natsorted(os.listdir(paths['rgb_raw']))
    # print(classNames)
    # print('type: ',type(classNames))
    classNames=classNames[class_idx:]

    for classname in classNames:
        
        
        video_dires=natsorted(glob(join(paths['rgb_raw'],classname)+'/*.mp4'))

        video_dires=video_dires[video_idx:]

        target_folder=join(paths['saved_dire'],classname)

        if not os.path.exists(target_folder):
            os.makedirs(target_folder)

        for idx,video_dire in enumerate(video_dires):

            print(video_dire)

            video_name=basename(video_dire)[:-4]

            features=extract_features(video_dire)

            target_dire=join(target_folder,'{}.npy'.format(video_name))

            np.save(target_dire,features)

            if (idx%10==0):
                print('{}: {} videos features extracted'.format(classname,idx))




  

        
        

        

    

        

    

1 Physical GPUs, 1 Logical GPUs


In [1]:
import tensorflow as tf
import tensorflow_hub as hub

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



import numpy as np
from glob import glob
import cv2 as cv
import os 
from os.path import join,basename
from natsort import natsorted 
import random



1 Physical GPUs, 1 Logical GPUs


2022-12-24 00:51:57.261033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 00:51:57.286644: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 00:51:57.286761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-24 00:51:57.287365: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [2]:
seq_len=64

In [3]:
test_dire='raw_dataset/Cook.Cleandishes/Cook.Cleandishes_p10_r00_v13_c06.mp4'

In [4]:
def sample_frames(video_dire):
    cap=cv.VideoCapture(video_dire)
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

    skip_frames=total_frames//seq_len


    frames=[]

    if skip_frames==0:

        diff_frame=seq_len-total_frames
        padd_arr=np.zeros((diff_frame,480,640,3),dtype=np.uint8)

        while cap.isOpened():
            
            sucess,frame=cap.read()
            
            if not sucess:
                break
            frames.append(frame)
        frames=np.concatenate((frames,padd_arr))
        return np.array(frames)


    if skip_frames==1:
        selected_frames=random.sample(range(total_frames),seq_len)
        selected_frames=sorted(selected_frames)
        for frame_number in selected_frames:
            cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
            sucess,frame=cap.read()
            frames.append(frame)
        return np.array(frames)

    
    if skip_frames>1:

        selected_frames=[i for i in range(0,total_frames,skip_frames)]

        if len(selected_frames) >seq_len:
            np.random.shuffle(selected_frames)
            selected_frames=selected_frames[:seq_len]
            selected_frames=sorted(selected_frames)
            for frame_number in selected_frames:
                cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
                sucess,frame=cap.read()
                frames.append(frame)
            
            return np.array(frames)


        else:
            for frame_number in selected_frames:
                
                cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
                sucess,frame=cap.read()
            
                frames.append(frame)

            return np.array(frames) 


In [5]:
frames=sample_frames(test_dire)

In [10]:
frame=frames[25]

In [11]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [12]:
crop_frame=crop_center_square(frame)

In [13]:
crop_frame.shape

(480, 480, 3)

: 

In [18]:
cv.imshow('',crop_frame)
cv.waitKey(0)
cv.destroyAllWindows()


QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to tar

In [19]:
crop2=crop_center_square(crop_frame)

In [20]:
cv.imshow('',crop2)
cv.waitKey(0)
cv.destroyAllWindows()


QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to target thread (0x55a7726d3ce0)

QObject::moveToThread: Current thread (0x55a7726d3ce0) is not the object's thread (0x55a772687040).
Cannot move to tar

In [41]:
model_input = tf.constant(frames, dtype=tf.float32)[tf.newaxis, ...]

In [39]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

In [42]:
i3d(model_input)['default'][0]

2022-12-23 20:07:05.397753: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.00G (1073741824 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-23 20:07:05.407041: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 1.00G (1073741824 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-23 20:07:05.407080: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 616.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-23 20:07:05.407089: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2022-12-23 20:07:05.420914: E tensorflow/stream

UnimplementedError: Graph execution error:

Detected at node 'RGB/inception_i3d/Conv3d_1a_7x7/conv_3d/convolution' defined at (most recent call last):
    File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/cvlab/.local/lib/python3.10/site-packages/tornado-6.2-py3.10-linux-x86_64.egg/tornado/platform/asyncio.py", lin

: 